# Refactored HistoryMatching workflow example

## 1. Set up

In [1]:
import torch

# imports from main
from autoemulate.history_matching_dashboard import HistoryMatchingDashboard

# imports from experimental
from autoemulate.experimental.emulators.gaussian_process.exact import (
    GaussianProcessExact,
)
from autoemulate.experimental.simulations.epidemic import Epidemic
from autoemulate.experimental.history_matching import HistoryMatching, HistoryMatchingWorkflow

### Simulate

Set up a Simulator and generate data.

In [2]:
simulator = Epidemic()
x = simulator.sample_inputs(10)
y = simulator.forward_batch(x)

Running simulations: 100%|██████████| 10/10 [00:00<00:00, 602.04it/s]

Successfully completed 10/10 simulations (100.0%)


### Train a GP

(this should be done with AutoEmulate obviously)

In [3]:
gp_pytorch = GaussianProcessExact(
        x,
        y,
    )
gp_pytorch.fit(x, y)

### Generate predictions

In [4]:
x = simulator.sample_inputs(5)
output = gp_pytorch.predict(torch.tensor(x, dtype=torch.float32))
pred_means, pred_vars = (
    output.mean.float().detach(),
    output.variance.float().detach(),
)

/var/folders/bj/kdwy1bhj3h728lr5xdj19yd40000gr/T/ipykernel_65903/1935492425.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = gp_pytorch.predict(torch.tensor(x, dtype=torch.float32))


## 2. HistoryMatching

Firstly, one can instantiate HistoryMatching without a simulator or an emulator. It can be used to calculate implausability for a given set of predictions.

In [5]:
# Define observed data with means and variances
observations = {"beta": (0.25, 0.05), "gamma": (0.1, 0.01)}

# Create history matcher
hm = HistoryMatching(
    observations=observations,
    threshold=3.0
)

implausability = hm.calculate_implausibility(pred_means, pred_vars)


Once implausability has been calculated, it can be used to identify NROY parameters.

In [6]:

hm.get_nroy(implausability, x)

tensor([[0.1587, 0.1554],
        [0.4181, 0.0351],
        [0.4302, 0.1927],
        [0.2121, 0.1181],
        [0.3121, 0.0561]])

## 3. Iterative HistoryMatchingWorkflow

We also have a separate class that implements an iterative sample-predict-evaluate workflow where in each wave:
- sample parameter values to test from the NROY space
    - at the start, NROY is the entire parameter space
    - use emulator to filter out implausible samples
- make predictions for the sampled parameters using the simulator
- refit the emulator using the simulated data

In [7]:
hmw = HistoryMatchingWorkflow(
    simulator=simulator,
    emulator=gp_pytorch,
    observations=observations,
    threshold=3.0
)

hmw.run(n_samples=40)

hmw.tested_params.shape, hmw.impl_scores.shape

Running simulations: 100%|██████████| 40/40 [00:00<00:00, 789.88it/s]

Successfully completed 40/40 simulations (100.0%)


(torch.Size([40, 2]), torch.Size([40, 2]))

## 4. Integration with dashboard

In [8]:
dashboard = HistoryMatchingDashboard(
    samples=hmw.tested_params,
    impl_scores=hmw.impl_scores,
    param_names=simulator.param_names,  
    output_names=simulator.output_names, 
    )

In [9]:
dashboard.display()

HTML(value='<h2>History Matching Dashboard</h2>')